# Calculations of PPVs for UWL cohort

16 Feb 2023

---

## Description

This notebook details the calculation of the diagnostic accuracy statistics from the cohort of Unexpected Weight Loss (UWL) patients and allows for comparison with the UK study

Statistics calculated include:
- PPVs for the UWL cohort for each subcohort defined by age group and sex
- PPVs for the UWL cohort for each subcohort defined by age group and pathology test group

These statistics are then calculated for each of the UWL cohorts defined solely in terms of 'Unintended', i.e., without the 'Intent unknown' category.

## 0.1. Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import re
from scipy.stats import binomtest
import seaborn as sns

sns.set()
%matplotlib inline

In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)

## 0.2. Load data

In [3]:
source_folder1 = 'M:/Working/DataAnalysis/CleanAndStructure/DataFiles'
source_folder2 = "M:/Working/AL/projects/weight_loss/outputs"
source_folder3 = "M:/Working/MDAP_Collaboration/data/original/NPS_Complete"
source_folder4 = "M:/Working/DataAnalysis/RCOLD/DataFiles"

filename_cohort = 'uwl_cohort_patron_300823.parquet'
filename_patient = "nps_patron_with_age_groups_Vic_ranges_300823.parquet" # includes the age group fields
filename_patient = 'nps_patron_with_age_groups_uk_ranges_201223.parquet'

filename_cohort = 'uwl_cohort_patron_nps_no_u18_300823.parquet'
filename_cohort = 'uwl_cohort_cleanest_201223.parquet'

filename_cancer = 'VAED_Cancers_coded.csv'

In [4]:
# the cohort of all weight related patients
#cohort_base = pd.read_parquet(f'{source_folder2}/{filename_cohort}')
cohort_base = pd.read_parquet(f'{source_folder2}/{filename_cohort}')

# patient table
#patient = pd.read_parquet(f'{source_folder2}/{filename_patient}')
patient = pd.read_parquet(f'{source_folder2}/{filename_patient}')

# filter to those patients in the cohort who do not have a weight increase
cohort = cohort_base.query('weight_increase == False')

#pathology = pd.read_csv(f'{source_folder2}/{filename_pathology}')
cancer = pd.read_csv(f'{source_folder4}/{filename_cancer}')

In [5]:
usi_uwl = cohort.query('usi_linked == True').query('uwl_flag == True').usi.unique()

In [6]:
# time difference between event and index date
time_differences = (
    cohort
    .groupby('usi')
    .apply(lambda g: g[g['index_case'] == True]['dte'].values[0] - g['dte'])
)

cohort['index_date_dt'] = time_differences.values

In [7]:
# standardize the USI values to be left zero-padded 10 digits
cohort = (
    cohort
    .query('usi in @usi_uwl')
    .assign(usi=lambda df_: df_.usi.astype(str).str.pad(width=10, side='left', fillchar='0'))
)

patient = (
    patient
   .assign(usi=lambda df_: df_.usi.astype(str).str.pad(width=10, side='left', fillchar='0'))
)

In [8]:
usi_uwl = cohort['usi'].unique()
patient_uwl = patient[patient['usi'].isin(usi_uwl)]

In [1]:
#cohort.shape, patient_uwl.shape, patient_uwl.usi.unique().shape

In [2]:
#cohort.shape, patient_uwl.shape, patient_uwl.usi.unique().shape

In [3]:
#cohort.usi.unique().shape

In [4]:
#cohort.columns

In [13]:
usi1 = cohort.query('uwl_flag == True').query('uwl_prediction == 1').usi.unique()
usi2 = cohort.query('uwl_flag == True').query('uwl_prediction == 2').usi.unique()
usi12 = cohort.query('uwl_flag == True').query('(uwl_prediction == 1) or (uwl_prediction == 2)').usi.unique()

In [5]:
#cohort1 = cohort.query('usi in @usi1')
#cohort2 = cohort.query('usi in @usi2')

#usi1.shape, usi2.shape, usi12.shape

In [6]:
# number of cancers
#cohort1.query('cancer_within_6months_index_date == True').usi.unique().shape, cohort2.query('cancer_within_6months_index_date == True').usi.unique().shape

In [7]:
#cohort1.query('cancer_within_6months_index_date == True').usi.unique().shape, cohort2.query('cancer_within_6months_index_date == True').usi.unique().shape

In [8]:
#cohort1.query('cancer_within_6months_index_date == True').usi.unique().shape[0] / usi1.shape[0] * 100, cohort2.query('cancer_within_6months_index_date == True').usi.unique().shape[0] / usi2.shape[0] * 100

In [9]:
#51 / 3616 * 100, 55 / 3903

In [10]:
# number of patients
#usi1.shape, usi2.shape, usi12.shape

In [11]:
# number of events
#cohort1.shape, cohort2.shape, cohort.shape

In [ ]:
# median time coverage of patients (median time between most recent and first event)
(
    cohort
    .groupby('usi', as_index=False)
    .dte.agg([np.min, np.max])
    .reset_index()
    .assign(time_difference=lambda df_: df_.amax-df_.amin)
    .time_difference.dt.days.median() / 365.25
)

In [12]:
# number of patients with at least one weight measurement
cohort.query('event_type == "observation"').query('event_subtype == "WEIGHT"').usi.unique().shape[0]

In [ ]:
# number of patients with at least two weight measurements in the two years before the index date
(
    cohort
    .query('two_years_before == True')
    .query('event_type == "observation"')
    .query('event_subtype == "WEIGHT"')
    .groupby('usi', as_index=False)
    .size()
    .query('size >= 2')
    .usi.unique().shape[0]
)

In [ ]:
# mean weight change in two years up to index date (for those patients with at least two weight measurements)
usi_double_weights = (
    cohort2
    .query('two_years_before == True')
    .query('event_type == "observation"')
    .query('event_subtype == "WEIGHT"')
    .groupby('usi', as_index=False)
    .size()
    .query('size >= 2')
    .usi.unique()
)

(
    cohort2
    .query('two_years_before == True')
    .query('event_type == "observation"')
    .query('event_subtype == "WEIGHT"')
    .query('usi in @usi_double_weights')
    .sort_values(by=['usi', 'dte'])
    .groupby('usi').agg(first_weight=('value', 'first'), last_weight=('value', 'last'))
    .assign(weight_difference=lambda df_: (df_.last_weight.astype(float) - df_.first_weight.astype(float)))
    .weight_difference.median()
)

## 0.3. Useful functions

In [26]:
from scipy.stats import norm

In [27]:
# for calculating Positive Predictive Values and confidence intervals
def z_value(alpha):
    return norm.ppf(1 - alpha/2)

def calculate_ppv(tp, fp):
    return tp / (tp + fp)

def calculate_seppv(tp, fp):
    ppv = calculate_ppv(tp, fp)
    se = np.sqrt(ppv * (1 - ppv) / (tp + fp))
    return se

def calculate_cippv(tp, fp, alpha):
    ppv = calculate_ppv(tp, fp)
    se = calculate_seppv(tp, fp)
    ci_l = ppv - z_value(alpha)*se
    ci_r = ppv + z_value(alpha)*se
    return ci_l, ci_r

In [28]:
def calculate_ppv_with_ci(tp, fp):
    p = tp / (tp + fp)
    b = binomtest(tp, tp+fp, p)
    ci_l, ci_r = b.proportion_ci()
    return p, ci_l, ci_r

In [29]:
def best_pathology_result(results):
    """
    Given a set of results, return the one that is either 
    within one month after the index date or if this does 
    not exist then the one that is the closest in the previous 
    3 months
    
    Args
    results (pd.DataFrame): dataframe with at least the following fields
        one_month_after
        one_months_before
        dte
        result
        
    Return
    best_result (pd.DataFrame): most representative pathology results
    """
    results_after = results[results['one_month_after'] == True]
    results_before = results[results['three_months_before'] == True]
    
    if results_after.shape[0] > 0:
        best_result = results_after.sort_values(by='index_date_dt').iloc[0:1]
    elif results_before.shape[0] > 0:
        best_result = results_before.sort_values(by='index_date_dt').iloc[0:1]
    else:
        best_result = results_after
        
    return best_result


def results_in_pathology_group(results, pathology_group):
    """
    Given a set of results and list of pathology test names
    determine if the results contains them all
    
    Args
    results (pd.DataFrame): dataframe with pathology data
    pathology_group (list of str): pathology test names
    
    Returns
    result_in_group (bool): True if results contains all the 
        result names
    """
    results_overlap = set(results['event_subtype'].values).intersection(pathology_group)
    results_in_group = (len(results_overlap) == len(pathology_group))
    return results_in_group


def calculate_cohort_counts(patients, groups=['age_group', 'gender_code'], ascending_order=[True, False]):
    """
    Calculate the numbers of cancer and non-cancer patients
    
    Args
    patients (pd.DataFrame): has columns usi, num_cancer_patients along with columns in groups
    groups (list of str): the columns that define the grouping of the data
    ascending_order (list of bool): ordering of the values for each of the columns in groups
    
    Returns
    ppv_results (pd.DataFrame): patients dataframe with ppvs calculated
    """
    cohort_counts = patients.groupby(groups)['cancer'].agg(num_patients=('size'), num_cancer=('sum'))
    
    # order the groups
    cohort_counts = cohort_counts.sort_values(by=groups, ascending=ascending_order)
    cohort_counts['num_no_cancer'] = cohort_counts['num_patients'] - cohort_counts['num_cancer']
    
    return cohort_counts

In [30]:
def results_in_pathology_group(results, pathology_group):
    """
    Given a set of results and list of pathology test names
    determine if the results contains them all
    
    Args
    results (pd.DataFrame): dataframe with pathology data
    pathology_group (list of str): pathology test names
    
    Returns
    result_in_group (bool): True if results contains all the 
        result names
    """
    results_overlap = set(results['event_subtype'].values).intersection(pathology_group)
    results_in_group = (len(results_overlap) == len(pathology_group))
    return results_in_group

## Cohort statistics

Create the 'Table 1' stats:
- Age group breakdown
- Sex breakdown
- Smoking status breakdown
- BMI breakdown
- Cancer within 6 months breakdown

In [31]:
# calculate the bmi for each of the patients
def calculate_bmi(df, index_date):
    """
    Given a patient, calculate the BMI value either using the explicit BMI
    or from HEIGHT and WEIGHT values were these are available. If information
    not available then return Null.
    
    Use the values closest to the index date
    """
    # time_difference: number of days to index date
    # so that we can pick the closest
    df = (
        df.copy()
        .assign(time_difference=(df.dte - index_date).dt.days.abs())
    )
    
    event_subtypes = df.event_subtype.values
    
    if 'BMI' in event_subtypes:
        return df.query('event_subtype == "BMI"').sort_values(by='time_difference').value.values[0]
    else:
        if ('HEIGHT' in event_subtypes) and ('WEIGHT' in event_subtypes):
            height = df.query('event_subtype == "HEIGHT"').sort_values(by='time_difference').value.values[0]
            weight = df.query('event_subtype == "WEIGHT"').sort_values(by='time_difference').value.values[0]
            return float(weight) / ((float(height) / 100)**2)
        else:
            return np.nan


In [32]:
usi_to_index_date = dict(cohort.query('index_case == True')[['usi', 'dte']].drop_duplicates().values)

In [33]:
cohort_obs = cohort.query('(six_months_before == True) or (three_months_after == True)')

In [34]:
usi_to_bmi_at_index = dict()

# for each of the patients find the bmi
for usi in cohort.usi.unique():
    df_temp = cohort_obs.query(f'usi == "{usi}"')
    index_date = usi_to_index_date[usi]
    bmi = calculate_bmi(df_temp, index_date)
    usi_to_bmi_at_index[usi] = float(bmi)

In [35]:
patient = (
    patient
    .assign(bmi_at_index_date=patient.usi.map(usi_to_bmi_at_index))
)

### Table 1 stats

In [36]:
# create table as input for PPV calculations:
# id, age, sex, smoking status, exposure, outcome (only have exposure=1 rows)
usi_to_outcome = dict(cohort[['usi', 'cancer_within_6months_index_date']].drop_duplicates().values)

In [13]:
cohort.query('uwl_flag == True').usi.unique().shape

In [14]:
cohort[['usi', 'cancer_within_6months_index_date']].drop_duplicates().shape

In [15]:
cohort[['usi', 'cancer_within_6months_index_date']].drop_duplicates().cancer_within_6months_index_date.sum()

In [16]:
55 / 3903 * 100

In [17]:
patient.query('usi in @usi_uwl').usi.unique().shape

In [42]:
# only those patients in the UWL cohort
patient_uwl = (
    patient.query('usi in @usi_uwl')
    .loc[:, ['usi', 'age_at_index_date', 'age_group_at_index_date', 
             'gender_code', 'smoking_status', 'bmi_at_index_date']]
    .assign(cancer_within_6_months=lambda df_: df_.usi.map(usi_to_outcome))
)

In [43]:
# for those patients with multiple records, order the smoking status as:
smoking_status_categories = ['Smoker', 'Ex smoker', 'Non smoker', 'Not Recorded']

In [44]:
bmi_range_values = [0, 18.5, 25, 30, 120]
bmi_range_labels = ['underweight', 'normal', 'overweight', 'obese']

In [45]:
patient_uwl_deduped = (
    patient_uwl
    .assign(smoking_status=pd.Categorical(patient_uwl.smoking_status.fillna('Not Recorded'), 
                                          categories=smoking_status_categories[::-1], ordered=True))
    .sort_values(by=['usi', 'smoking_status'], 
                 ascending=[True, False])
    .groupby('usi')
    .first()
    .query('age_at_index_date >= 18')
    .assign(bmi_range=lambda df_: 
            pd.cut(df_.bmi_at_index_date, bins=bmi_range_values, labels=bmi_range_labels).astype(str).replace('nan', 'missing'))
    .reset_index()
)

In [18]:
patient_uwl_usi = patient_uwl_deduped.usi.unique()
patient_uwl_usi.shape

In [19]:
cohort_usi = cohort.usi.unique()
cohort_usi.shape

In [48]:
cohort_not_patient = [usi for usi in cohort_usi if usi not in patient_uwl_usi]

In [20]:
len(cohort_not_patient)

In [50]:
#cohort.query('usi in @cohort_not_patient').uwl_label.value_counts()

In [21]:
cohort.usi.unique().shape

In [22]:
patient_uwl.usi.unique().shape

In [23]:
cohort.query('uwl_flag == True').usi.unique().shape

In [54]:
#cohort.query('uwl_label == True').usi.unique().shape

In [55]:
#cohort.query('uwl_label == 3').usi.unique().shape

In [24]:
num_uwl_patients = patient_uwl_deduped.shape[0]
num_uwl_patients

In [25]:
patient_uwl_deduped.groupby('age_group_at_index_date').size()

In [58]:
ages = patient_uwl_deduped.age_at_index_date

In [26]:
patient_uwl_deduped.groupby('age_group_at_index_date').size() / num_uwl_patients * 100

In [27]:
patient_uwl_deduped.groupby('gender_code').size() / num_uwl_patients * 100

In [28]:
patient_uwl_deduped.groupby('gender_code').size()

In [29]:
#patient_uwl_deduped.groupby('smoking_status').size()

In [30]:
#patient_uwl_deduped.groupby('smoking_status').size() / num_uwl_patients * 100

In [31]:
#patient_uwl_deduped.groupby('smoking_status').size() / num_uwl_patients * 100

In [32]:
#patient_uwl_deduped.groupby('smoking_status').size() #/ num_uwl_patients * 100

In [33]:
#patient_uwl_deduped.groupby('bmi_range').size() / num_uwl_patients * 100

In [34]:
#patient_uwl_deduped.groupby('bmi_range').size()# / num_uwl_patients * 100

In [35]:
num_uwl_patients

In [36]:
patient_uwl_deduped.groupby('cancer_within_6_months').size() / num_uwl_patients * 100

In [37]:
patient_uwl_deduped.groupby('cancer_within_6_months').size()

In [71]:
cancer_uwl = (
    cancer
    .assign(usi=cancer.usi.astype(str).str.pad(width=10, side='left', fillchar='0'))
    .assign(incidence_dte=pd.to_datetime(cancer.incidence_dte))
    .query('usi in @usi_uwl')
    .assign(uwl_index_date=lambda df_: df_.usi.map(usi_to_index_date))
    .assign(days_from_index=lambda df_: (df_.incidence_dte-df_.uwl_index_date).dt.days)
    .assign(six_months_from_index=lambda df_: (df_.days_from_index < 186) & (df_.days_from_index > 0))
)

In [72]:
usi_uwl_cancer1 = patient_uwl_deduped.query('cancer_within_6_months == True').reset_index().usi

In [38]:
usi_uwl_cancer1.values.shape

In [39]:
usi_uwl_cancer1.values.shape

## Positive predictive values

In [75]:
def diagnostic_stats(patients, 
                     stratify_by=['age', 'sex'], 
                     cumulative_count_by=['age'], 
                     outcome=['cancer']):
    """
    Calculate diagnostic stats for a patient table: true positives, false positives, PPV
    """
    cumulative_stratify_by = list(set(stratify_by) - set(cumulative_count_by))
    sort_order = [True for _ in range(len(cumulative_stratify_by))]
    
    df_results = (
        patients
        .groupby(stratify_by+outcome, as_index=False)
        .size()
        .pivot(index=stratify_by, columns=outcome, values='size')
        .reset_index()
        .rename(columns={True: 'TP', False: 'FP'})
        .assign(PPV=lambda df_: df_.TP / (df_.TP + df_.FP) * 100)
        .sort_values(by=stratify_by[::-1])
        .assign(FP_cumulative=lambda df_: df_.sort_values(by=stratify_by[::-1], 
                                                          ascending=sort_order + [False])
                .groupby(cumulative_stratify_by)['FP'].cumsum())
        .assign(TP_cumulative=lambda df_: df_.sort_values(by=stratify_by[::-1], 
                                                          ascending=sort_order + [False])
                .groupby(cumulative_stratify_by)['TP'].cumsum())
        .assign(PPV_cumulative=lambda df_: df_.TP_cumulative / (df_.TP_cumulative + df_.FP_cumulative) * 100)
    )
    
    return df_results

### Overall

In [ ]:
df_results = (
    patient_uwl_deduped
    .groupby(['age_group_at_index_date', 'cancer_within_6_months'], as_index=False)
    .size()
    .pivot(index='age_group_at_index_date', columns='cancer_within_6_months', values='size')
    .rename(columns={True: 'TP', False: 'FP'})
    .reset_index()
    .iloc[1:, :]
    .assign(FP_cumulative=lambda df_: df_.sort_values(by='age_group_at_index_date', ascending=False).FP.cumsum()[::-1].values)
    .assign(TP_cumulative=lambda df_: df_.sort_values(by='age_group_at_index_date', ascending=False).TP.cumsum()[::-1].values)
    .assign(PPV_cumulative=lambda df_: df_.TP_cumulative / (df_.TP_cumulative + df_.FP_cumulative) * 100)
).reset_index().iloc[:, 1:]

df_results_ppv = (
    pd.DataFrame(df_results.apply(lambda row: calculate_ppv_with_ci(row['TP_cumulative'], row['FP_cumulative']), axis=1)
                 .tolist())
    .rename(columns={0: 'ppv', 1: 'ppv_ci_l', 2: 'ppv_ci_r'})*100
)

df_results = pd.concat([df_results, df_results_ppv], axis=1).reset_index()
df_results

### By age and gender

In [77]:
df_results = (
    diagnostic_stats(patient_uwl_deduped, 
                     stratify_by=['age_group_at_index_date', 'gender_code'], 
               #      stratify_by=['age_group_at_index_date'],
                     cumulative_count_by=['age_group_at_index_date'],
                     outcome=['cancer_within_6_months'])
    .reset_index()
    .iloc[:, 1:]
)

df_results_ppv = (
    pd.DataFrame(df_results.apply(lambda row: calculate_ppv_with_ci(row['TP_cumulative'], 
                                                                                 row['FP_cumulative']), axis=1)
                 .tolist())
    .rename(columns={0: 'ppv', 1: 'ppv_ci_l', 2: 'ppv_ci_r'})*100
)


df_results = pd.concat([df_results, df_results_ppv], axis=1).reset_index().sort_values(by='gender_code')

In [78]:
df_age_sex = df_results

In [ ]:
df_age_sex

## 1. Positive predictive values

- Cancer within next 6 months from index date
- Patient experiences UWL event, probability of cancer within next 6 months

- By age, sex
- By pathology result, age, sex

In [ ]:
df_age_sex

In [ ]:
# change the labels of the groups
ticklabels = df_results['gender_code'].astype(str) + \
', >=' + \
df_results['age_group_at_index_date'].str.replace('+', '').str.split('-').apply(lambda l: l[0])

# plot these out
plt.figure(figsize=(12, 8))
plt.hlines(y=df_results.index, 
           xmin=df_results['ppv_ci_l'], 
           xmax=df_results['ppv_ci_r'], alpha=0.25)

sns.scatterplot(data=df_results, x='ppv_ci_l', y='index', color='b', alpha=0.25)
sns.scatterplot(data=df_results, x='ppv_ci_r', y='index', color='b', alpha=0.25)
sns.scatterplot(data=df_results, x='ppv', y='index', color='k')

plt.yticks(ticks=df_results['index'].values, labels=ticklabels)
plt.xlabel('Positive Predictive Values')
plt.ylabel('')
plt.xlim(0, 10)

plt.title('Positive predictive values of UWL patients by age group and gender')
plt.show()

In [ ]:
# change the labels of the groups
ticklabels = df_results['gender_code'].astype(str) + \
', >=' + \
df_results['age_group_at_index_date'].str.replace('+', '').str.split('-').apply(lambda l: l[0])

# plot these out
plt.figure(figsize=(12, 8))
plt.hlines(y=df_results.index, 
           xmin=df_results['ppv_ci_l'], 
           xmax=df_results['ppv_ci_r'], alpha=0.25)

sns.scatterplot(data=df_results, x='ppv_ci_l', y='index', color='b', alpha=0.25)
sns.scatterplot(data=df_results, x='ppv_ci_r', y='index', color='b', alpha=0.25)
sns.scatterplot(data=df_results, x='ppv', y='index', color='k')

plt.yticks(ticks=df_results['index'].values, labels=ticklabels)
plt.xlabel('Positive Predictive Values')
plt.ylabel('')
plt.xlim(0, 10)

plt.title('Positive predictive values of UWL patients by age group and gender')
plt.show()

## 1.1 PPVs with smoking status

In [ ]:
smoker = ['Smoker', 'Ex smoker']

patient_uwl_deduped_smoking = (
    patient_uwl_deduped
    .query('smoking_status != "Not Recorded"')
    .assign(smoker=lambda df_: df_.smoking_status.isin(smoker))
)
patient_uwl_deduped_smoking

In [ ]:
patient_uwl_deduped_smoking.usi.unique().shape[0]

### By age

In [85]:
df_results = (
    diagnostic_stats(patient_uwl_deduped_smoking, 
                     stratify_by=['age_group_at_index_date', 'smoker'], 
                     cumulative_count_by=['age_group_at_index_date'],
                     outcome=['cancer_within_6_months'])
    .reset_index()
    .iloc[:, 1:]
)

df_results_ppv = (
    pd.DataFrame(df_results.apply(lambda row: calculate_ppv_with_ci(row['TP_cumulative'], 
                                                                                 row['FP_cumulative']), axis=1)
                 .tolist())
    .rename(columns={0: 'ppv', 1: 'ppv_ci_l', 2: 'ppv_ci_r'})*100
)


df_results = (
    pd.concat([df_results, df_results_ppv], axis=1)
    .reset_index()
    .sort_values(by=['smoker', 'age_group_at_index_date'])
)

In [ ]:
df_results

In [ ]:
df_results = (
    patient_uwl_deduped_smoking
    .groupby(['age_group_at_index_date', 'cancer_within_6_months'], as_index=False)
    .size()
    .pivot(index='age_group_at_index_date', columns='cancer_within_6_months', values='size')
    .rename(columns={True: 'TP', False: 'FP'})
    .reset_index()
    .iloc[1:, :]
    .assign(FP_cumulative=lambda df_: df_.sort_values(by='age_group_at_index_date', ascending=False).FP.cumsum()[::-1].values)
    .assign(TP_cumulative=lambda df_: df_.sort_values(by='age_group_at_index_date', ascending=False).TP.cumsum()[::-1].values)
    .assign(PPV_cumulative=lambda df_: df_.TP_cumulative / (df_.TP_cumulative + df_.FP_cumulative) * 100)
).reset_index().iloc[:, 1:]

df_results_ppv = (
    pd.DataFrame(df_results.apply(lambda row: calculate_ppv_with_ci(row['TP_cumulative'], 
                                                                                 row['FP_cumulative']), axis=1)
                 .tolist())
    .rename(columns={0: 'ppv', 1: 'ppv_ci_l', 2: 'ppv_ci_r'})*100
)

df_results = pd.concat([df_results, df_results_ppv], axis=1).reset_index()
df_results

### By age and gender

In [88]:
df_results = (
    diagnostic_stats(patient_uwl_deduped_smoking, 
                     stratify_by=['age_group_at_index_date', 'gender_code', 'smoker'], 
                     cumulative_count_by=['age_group_at_index_date'],
                     outcome=['cancer_within_6_months'])
    .reset_index()
    .iloc[:, 1:]
)

df_results_ppv = (
    pd.DataFrame(df_results.apply(lambda row: calculate_ppv_with_ci(row['TP_cumulative'], 
                                                                                 row['FP_cumulative']), axis=1)
                 .tolist())
    .rename(columns={0: 'ppv', 1: 'ppv_ci_l', 2: 'ppv_ci_r'})*100
)


df_results = (
    pd.concat([df_results, df_results_ppv], axis=1)
    .reset_index()
    .sort_values(by=['smoker', 'gender_code', 'age_group_at_index_date'])
)

In [ ]:
df_smokers = df_results.query('smoker==True')
df_smokers.reset_index()

In [ ]:
df_non_smokers = df_results.query('smoker==False')
df_non_smokers.reset_index()

In [91]:
df_age_sex = df_age_sex.assign(group='UWL')
df_smokers = df_smokers.assign(group='UWL in smokers')
df_non_smokers = df_non_smokers.assign(group='UWL in non-smokers')

In [92]:
age_groups = ['40 - 49', '50 - 59', '60 - 69', '70 - 79', '80+']

In [93]:
df_smokers_vic = (
    df_smokers
    .query('age_group_at_index_date in @age_groups')
    .reset_index()
    .loc[:, ['age_group_at_index_date', 'gender_code', 'ppv', 'ppv_ci_l', 'ppv_ci_r', 'group']]
    .assign(order=range(10))
)

df_non_smokers_vic = (
    df_non_smokers
    .query('age_group_at_index_date in @age_groups')
    .reset_index()
    .loc[:, ['age_group_at_index_date', 'gender_code', 'ppv', 'ppv_ci_l', 'ppv_ci_r', 'group']]
    .assign(order=range(10))
)

df_age_sex_vic = (
    df_age_sex
    .query('age_group_at_index_date in @age_groups')
    .reset_index()
    .loc[:, ['age_group_at_index_date', 'gender_code', 'ppv', 'ppv_ci_l', 'ppv_ci_r', 'group']]
    .assign(order=range(10))
)

In [94]:
df_uwl_vic = pd.concat([df_age_sex_vic, df_smokers_vic, df_non_smokers_vic]).assign(country='AU')

In [95]:
df_age_sex_uk = (
    pd.DataFrame(data={'age_group_at_index_date': ['40 - 50', '50 - 59', '60 - 69', '70 - 79', '80+']*2, 
                                   'gender_code': ['Female', 'Female', 'Female', 'Female', 'Female', 
                                                   'Male', 'Male', 'Male', 'Male', 'Male'], 
                                   'ppv': [1.26, 1.47, 1.71, 1.86, 1.83, 2.54, 2.98, 3.65, 4.35, 4.58], 
                                   'ppv_ci_l': [1.14, 1.32, 1.53, 1.65, 1.56, 2.34, 2.74, 3.34, 3.94, 3.98], 
                                   'ppv_ci_r': [1.40, 1.63, 1.90, 2.09, 2.15, 2.76, 3.24, 3.98, 4.78, 5.25]})
    .assign(group='UWL', country='UK').reset_index()
)

df_smokers_uk = (
    pd.DataFrame(data={'age_group_at_index_date': ['40 - 50', '50 - 59', '60 - 69', '70 - 79', '80+']*2, 
                                   'gender_code': ['Female', 'Female', 'Female', 'Female', 'Female', 
                                                   'Male', 'Male', 'Male', 'Male', 'Male'], 
                                   'ppv': [1.31, 1.59, 1.90, 2.10, 2.12, 2.64, 3.06, 3.70, 4.34, 4.60], 
                                   'ppv_ci_l': [1.04, 1.27, 1.50, 1.59, 1.42, 2.26, 2.62, 3.15, 3.62, 3.51], 
                                   'ppv_ci_r': [1.61, 1.97, 2.37, 2.70, 3.03, 3.06, 3.55, 4.31, 5.16, 5.91]})
    .assign(group='UWL in smokers', country='UK').reset_index()
)

df_non_smokers_uk = (
    pd.DataFrame(data={'age_group_at_index_date': ['40 - 50', '50 - 59', '60 - 69', '70 - 79', '80+']*2, 
                                   'gender_code': ['Female', 'Female', 'Female', 'Female', 'Female', 
                                                   'Male', 'Male', 'Male', 'Male', 'Male'], 
                                   'ppv': [0.73, 0.90, 1.02, 1.15, 1.16, 1.58, 1.97, 2.38, 2.78, 3.38], 
                                   'ppv_ci_l': [0.57, 0.70, 0.79, 0.89, 0.87, 1.24, 1.56, 1.88, 2.17, 2.59], 
                                   'ppv_ci_r': [0.92, 1.14, 1.29, 1.46, 1.51, 1.97, 2.46, 2.96, 3.50, 4.34]})
    .assign(group='UWL in non-smokers', country='UK').reset_index()
)

In [96]:
df_uwl_uk = pd.concat([df_age_sex_uk, df_smokers_uk, df_non_smokers_uk]).assign(order=lambda df_: df_.index+0.25).iloc[:, 1:]

In [97]:
df_uwl_uk_vic = pd.concat([df_uwl_uk, df_uwl_vic])

In [ ]:
df_uwl_uk_vic.head(3)

In [99]:
ticklabels = df_uwl_uk_vic['age_group_at_index_date'].astype(str).values + ' years, ' + df_uwl_uk_vic['gender_code'].astype(str).values


In [ ]:
df_uwl_uk_vic.head(3)

In [101]:
yticklabels = df_uwl_uk_vic.head(10).gender_code + ', ' + df_uwl_uk_vic.head(10).age_group_at_index_date

In [ ]:
df_uwl_uk_vic.query('country == "AU"')

In [ ]:
sns.set_style('white')
g = sns.FacetGrid(data=df_uwl_uk_vic, col='group', height=5, aspect=0.85)
g.map(sns.scatterplot, 'ppv', 'order', 'country')

for col_val, ax in g.axes_dict.items():
    df_temp = df_uwl_uk_vic.query(f'group == "{col_val}"')
    
    ax.hlines(xmin=df_temp.query('country=="UK"').ppv_ci_l, 
              xmax=df_temp.query('country=="UK"').ppv_ci_r, 
              y=df_temp.query('country=="UK"').order)
    
    ax.hlines(xmin=df_temp.query('country=="AU"').ppv_ci_l, 
              xmax=df_temp.query('country=="AU"').ppv_ci_r, 
              y=df_temp.query('country=="AU"').order, color='r')
    
g.set_axis_labels('Positive Predictive Value', '')
g.set_titles(col_template="{col_name}")
g.add_legend()
g.set(yticks=np.arange(0,10) + 0.125)
g.set_yticklabels(yticklabels)

plt.show()

## 1.2. PPVs with abnormal test result values

Test results that are considered abnormal by BN
- albumin: low
- platelets: raised (abnormal_high)
- calcium: raised (high)
- white cell: raised (high, abnormal_high)
- crp: raised (elevated)
- haemaglobin: low (low)
- alp: raised (abnormal_high, high)
- bilirubin: raised (high, abnormal_high)
- esr: raised (high)
- creatinine: raised (high)

### By age, gender and pathology result value

Consider the following combinations of pathology test results (within period of 3 months before to 1 month after index date):
- Hb + CRP + Plt + Alb
- Hb + CRP + Plt
- Alb + CRP
- Hb + CRP
- Plt
- CRP
- Alb
- Hb

And the following age groups:
- 40 - 59 years
- 60 - 79 years
- 80+ years

Calculate PPV for each subset of patients in UWL for which all data available (i.e., no imputation). Why is this reasonable?

For each patient and pathology result:
- If there is a result in the 1 month after the index date, use this
- Otherwise use the one closest to the index date

## 1.3. Calculate PPVs for cancer based on UWL and abnormal path tests

For each pathology test grouping and age group and sex:
- calculate number of patients with cancer
- total number of patients in cohort with cancer
- ppv
- confidence intervals of ppv

In [104]:
# define the different pathology groups
path_group1 = ['HAEMOGLOBIN']
path_group2 = ['ALBUMIN']
path_group3 = ['CRP']
path_group4 = ['PLATELETS']
path_group5 = ['HAEMOGLOBIN', 'CRP']
path_group6 = ['ALBUMIN', 'CRP']
path_group7 = ['HAEMOGLOBIN', 'CRP', 'PLATELETS']
path_group8 = ['HAEMOGLOBIN', 'CRP', 'PLATELETS', 'ALBUMIN']

path_groups = [path_group1, 
               path_group2, 
               path_group3, 
               path_group4, 
               path_group5, 
               path_group6, 
               path_group7, 
               path_group8]

path_group_names = ['+'.join(path_group).lower() for path_group in path_groups]

In [ ]:
cohort.query('usi in @patient_uwl_usi').query('event_type == "pathology"').event_subtype.value_counts()

In [ ]:
cohort.event_type.value_counts()

In [ ]:
cohort.query('usi in @patient_uwl_usi').query('event_type == "pathology"').groupby(['event_subtype', 'value3']).size()

In [108]:
pathology = cohort.query('event_type == "pathology"')

In [ ]:
pathology.shape

In [ ]:
num_uwl_patients

In [ ]:
pathology.usi.unique().shape

In [112]:
# filter to those path results that are three months prior to one month after the index date
pathology_within_window = pathology.query('usi in @patient_uwl_usi').query('(three_months_before == True) or (one_month_after == True)')
pathology_within_window2 = pathology.query('(three_months_before == True) or (three_months_after == True)')

In [ ]:
pathology_within_window[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts()

In [ ]:
pathology_within_window[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts()

In [ ]:
pathology_within_window[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts() / num_uwl_patients * 100

In [117]:
#pathology_within_window2[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts() / num_uwl_patients * 100

In [ ]:
# how many patients have at least one of each type?
pathology_within_window[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts()

In [ ]:
# how many patients have at least one of each type?
pathology_within_window[['usi', 'event_subtype']].drop_duplicates()['event_subtype'].value_counts() / num_uwl_patients * 100

In [ ]:
(
    pathology_within_window
    .loc[:, ['usi', 'event_subtype', 'value3']]
    .drop_duplicates()
    .groupby(['event_subtype', 'value3'])
    .size()
)

In [ ]:
(
    pathology_within_window
    .loc[:, ['usi', 'event_subtype', 'value3']]
    .drop_duplicates()
    .groupby(['event_subtype', 'value3'])
    .size() / num_uwl_patients * 100
)

In [ ]:
# proportion of path results that are abnormal within each test (in three months prior to one month after)
pathology_ranges = (
    pathology_within_window
    .groupby(['event_subtype', 'value3'], as_index=False)
    .agg('size')
    .rename(columns={'size': 'num_measurements'})
    .assign(measurements_perc=lambda df_: df_.num_measurements 
            / df_.groupby('event_subtype')['num_measurements'].transform('sum') * 100)
)

pathology_ranges

In [ ]:
# proportion of path results that are abnormal within each test
pathology_ranges = (
    pathology
    .groupby(['event_subtype', 'value3'], as_index=False)
    .agg('size')
    .rename(columns={'size': 'num_measurements'})
    .assign(measurements_perc=lambda df_: df_.num_measurements 
            / df_.groupby('event_subtype')['num_measurements'].transform('sum') * 100)
)

pathology_ranges

In [124]:
time_period_vars = ['five_years_before', 
                    'two_years_before', 
                    'one_year_before', 
                    'six_months_before', 
                    'three_months_before', 
                    'one_month_before', 
                    'at_index_date',
                    'one_month_after',
                    'three_months_after',
                    'six_months_after', 
                    'one_year_after', 
                    'two_years_after',
                    'five_years_after']

In [125]:
# how many test results are there by time period?
pathology_times = pathology.query('usi in @patient_uwl_usi')[['usi', 'dte', 'event_subtype', 'value', 'usi_linked'] + time_period_vars]

In [126]:
# convert to tidy form to make calculations easier
pathology_tidy = (
    pd.melt(pathology_times, 
            id_vars=['usi', 'dte', 'event_subtype', 'value', 'usi_linked'], 
            value_vars=time_period_vars, 
            value_name='in_time_period', 
            var_name='time_period')
    .assign(time_period=lambda df_: pd.Categorical(df_.time_period, time_period_vars))
)

In [127]:
# number of pathology tests by time_period
pathology_test_time_period = (
    pathology_tidy
    .query('in_time_period == True')
    .groupby(['event_subtype', 'time_period'], as_index=False)
    .size()
)

In [ ]:
# pivot table showing number of patients with at least n weight measurements for each of the time periods (1 <= n <= 5)
time_period_count_pivot = pathology_test_time_period.pivot(index='event_subtype', 
                                                           columns='time_period', 
                                                           values='size').T

time_period_count_pivot

In [ ]:
plt.figure(figsize=(12, 8))
ax = sns.heatmap(time_period_count_pivot.T, annot=True, fmt='d')
plt.show()

In [130]:
# number of patients that have k of each test within time period
pathology_tidy_small = pathology_tidy[['usi', 'event_subtype', 'usi_linked', 'time_period', 'in_time_period']]
pathology_tidy_small = pathology_tidy_small.query('in_time_period == True')

In [ ]:
pathology_tidy_small

In [132]:
usi_test_time_period_counts = pathology_tidy_small.groupby(['usi', 'event_subtype', 'time_period'], as_index=False).size()

In [133]:
# for each time period and size how many patients are there?
time_period_size_counts = (
    usi_test_time_period_counts
    .groupby(['event_subtype', 
              'time_period', 
              'size'], as_index=False)
    .count()
    .query('size > 0')
    .rename(columns={'usi': 'num_usi'})
)

In [134]:
# sort the data and calculate number of usi in time period with at least size n
time_period_size_counts['cumulative_count'] = (
    time_period_size_counts
    .sort_values(by=['event_subtype', 'time_period', 'size'], ascending=[True, True, False])
    .groupby('time_period')['num_usi']
    .cumsum()
)

In [ ]:
# pivot table showing number of patients with at least n weight measurements for each of the time periods (1 <= n <= 5)
time_period_count_pivot = (
    time_period_size_counts[['event_subtype', 'time_period', 
                             'size', 'cumulative_count']]
    .query('size < 6')
    .pivot(index='time_period', columns=['event_subtype', 'size'], values='cumulative_count')
    .T
)

time_period_count_pivot

In [136]:
pathology = pathology.assign(value3=lambda df_: df_.value3.str.lower())

In [137]:
# filter to abnormal test results
tests = ['ALBUMIN', 
         'ALP', 
         'BILIRUBIN', 
         'CALCIUM', 
         'CRP', 
         'CREATININE', 
         'ESR', 
         'HAEMOGLOBIN', 
         'PLATELETS', 
         'LEUCOCYTES']

abnormal_codes = [['low'], 
                  ['abnormal_high', 'high'], 
                  ['abnormal_high', 'high'], 
                  ['high'],  ['elevated'], 
                  ['high'], 
                  ['high'], 
                  ['low'], 
                  ['high'], 
                  ['high', 'abnormal_high']]

test_to_code = dict(zip(tests, abnormal_codes))

# for each of the pathology tests above, filter to those with abnormal values
path_abnormals = []

for test, codeset in test_to_code.items():
    path_test_abnormal = pathology.query(f'event_subtype == "{test}"').query(f"value3 == @codeset")
    path_abnormals.append(path_test_abnormal)
    
pathology_abnormal = pd.concat(path_abnormals)

In [ ]:
(
    pathology_abnormal
    .query('(three_months_before == True) or (one_month_after == True)')
    .loc[:, ['usi', 'event_subtype', 'value3']]
    .drop_duplicates()
    .event_subtype
    .value_counts()# / 5603 * 100
)

In [ ]:
pathology_abnormal['event_subtype'].value_counts()

In [141]:
# how many test results are there by time period?
pathology_times = (
    pathology_abnormal[['usi', 'dte', 'event_subtype', 'value', 'usi_linked'] + time_period_vars]
)

In [142]:
# convert to tidy form to make calculations easier
pathology_tidy = pd.melt(pathology_times, 
                         id_vars=['usi', 'dte', 'event_subtype', 'value', 'usi_linked'], 
                         value_vars=time_period_vars, 
                         value_name='in_time_period', 
                         var_name='time_period')

# convert time period to categorical based on time ordering
pathology_tidy['time_period'] = pd.Categorical(pathology_tidy['time_period'], time_period_vars)

In [143]:
# number of patients that have k of each test within time period
pathology_tidy_small = pathology_tidy[['usi', 'event_subtype', 'usi_linked', 'time_period', 'in_time_period']]
pathology_tidy_small = pathology_tidy_small.query('in_time_period == True').query('usi_linked == True')

In [144]:
# number of events for each usi and time period?
usi_test_time_period_counts = (
    pathology_tidy_small
    .groupby(['usi', 'event_subtype', 'time_period'], as_index=False)
    .size()
)

In [145]:
# for each time period and size how many patients are there?
time_period_size_counts = (
    usi_test_time_period_counts
    .groupby(['event_subtype', 'time_period', 'size'], as_index=False)
    .count()
    .query('size > 0')
)

In [146]:
# sort the data and calculate number of usi in time period with at least size n
time_period_size_counts['cumulative_count'] = (
    time_period_size_counts
    .sort_values(by=['event_subtype', 'time_period', 'size'], 
                 ascending=[True, True, False])
    .groupby(['event_subtype', 'time_period'])['usi']
    .cumsum()
)

In [ ]:
# pivot table showing number of patients with at least n weight measurements for each of the time periods (1 <= n <= 5)
time_period_count_pivot = (
    time_period_size_counts[['event_subtype', 'time_period', 'size', 'cumulative_count']]
    .query('size < 6')
    .pivot(index='time_period', 
           columns=['event_subtype', 'size'], 
           values='cumulative_count')
    .T
)

time_period_count_pivot

In [ ]:
plt.figure(figsize=(12, 8))
ax = sns.heatmap(time_period_count_pivot.query('size == 1'), annot=True, fmt='d')
plt.show()

In [149]:
path_group1 = ['HAEMOGLOBIN']
path_group2 = ['ALBUMIN']
path_group3 = ['CRP']
path_group4 = ['PLATELETS']
path_group5 = ['HAEMOGLOBIN', 'CRP']
path_group6 = ['ALBUMIN', 'CRP']
path_group7 = ['HAEMOGLOBIN', 'CRP', 'PLATELETS']
path_group8 = ['HAEMOGLOBIN', 'CRP', 'PLATELETS', 'ALBUMIN']

path_group9 = ['CALCIUM']
path_group10 = ['CREATININE']
path_group11 = ['ALP']
path_group12 = ['BILIRUBIN']
path_group13 = ['LEUCOCYTES']
path_group14 = ['ESR']

path_groups = [path_group1, 
               path_group2, 
               path_group3, 
               path_group4, 
               path_group5, 
               path_group6, 
               path_group7, 
               path_group8, 
               path_group9, 
               path_group10, 
               path_group11, 
               path_group12, 
               path_group13, 
               path_group14]

path_group_names = ['_'.join(path_group).lower() for path_group in path_groups]

In [ ]:
pathology_abnormal.columns

In [151]:
# find the closest abnormal result for each test for each patient
pathology_abnormal_closest = pathology_abnormal.groupby(['usi', 'event_subtype'], as_index=False).apply(lambda g: best_pathology_result(g))
pathology_abnormal_closest = pathology_abnormal_closest.reset_index().iloc[:, 2:]

In [ ]:
pathology_abnormal_closest['event_subtype'].value_counts()

In [40]:
# for each of the pathology gruops and patients, create a flag for whether the event is part of this group
pathology_abnormal_closest.shape, pathology_abnormal_closest[['usi', 'event_subtype']].drop_duplicates().shape

In [154]:
# how many path results for each patient
pathology_small = pathology_abnormal_closest[['usi', 'dte', 'event_subtype', 'value3', 'usi_linked', 'index_date_dt']]

In [155]:
path_group = path_groups[0]

In [ ]:
pathology_small.groupby('usi', as_index=False)['event_subtype'].transform(lambda events: set(events) >= set(path_group))

In [ ]:
for path_group, path_group_label in zip(path_groups, path_group_names):
    results = pathology_small.groupby('usi', as_index=False)['event_subtype'].transform(
        lambda events: set(events) >= set(path_group))
    pathology_small[path_group_label] = results

In [ ]:
# how many patients in each group
(pathology_small.groupby('usi', as_index=False)[path_group_names].sum()[path_group_names] > 0).sum()

In [183]:
pathology_small.to_csv(f'{source_folder1}/pathology_index_values_all_230224.csv', index=False)

### 1.4. Calculate for age, pathology test group

In [160]:
usi_cancer = cohort.query('cancer_within_6months_index_date == True').usi.unique()
usi_non_cancer = cohort.query('cancer_within_6months_index_date == False').usi.unique()

In [161]:
dfs = []

# for each pathology group calculate the number of cancer patients and total number of patients
for n in range(len(path_groups)):
    usi_path_group = pathology_small.query(f'{path_group_names[n]} == True').usi.unique()
    
    counts_by_age = (
        patient
        .query('usi in @usi_path_group')
        .assign(cancer=False)
        .assign(cancer=lambda df_: df_.usi.isin(usi_cancer))    
        .sort_values(by=['age_group2_at_index_date'])
        .groupby(['age_group2_at_index_date'])['cancer']
        .agg(['sum', 'size'])
        .reset_index()
        .rename(columns={'sum': 'num_cancer', 'size': 'num_patients'})
        .assign(test_group=path_group_names[n])
    )
    dfs.append(counts_by_age)

# concate all the test groups together
cohort_counts = (
    pd.concat(dfs)
    .loc[:, ['test_group', 'age_group2_at_index_date', 'num_cancer', 'num_patients']]
    .reset_index()
    .iloc[:, 1:]
    .assign(test_group=lambda df_: pd.Categorical(df_.test_group, categories=path_group_names))
    .query('num_patients > 0')
    .sort_values(by=['age_group2_at_index_date', 'test_group'])
    .reset_index()
    .iloc[:, 1:]
)

# calculate the PPVs
cohort_ppv = (
    pd.DataFrame(
        cohort_counts
        .apply(lambda r: calculate_ppv_with_ci(r['num_cancer'], r['num_patients'] - r['num_cancer']), axis=1)
        .to_list(), columns=['ppv', 'ppv_ci_l', 'ppv_ci_r']
    ) * 100
)

# combine with the cohort counts
age_test_ppv = (
    pd.concat([cohort_counts, cohort_ppv], axis=1)
    .sort_values(by=['age_group2_at_index_date', 'test_group'], ascending=[True, False])
    .reset_index()
    .iloc[:, 1:]
    .query('age_group2_at_index_date != "<40"')
)

In [162]:
# subset of tests that appear in paper
path_groups_small = ['haemoglobin_crp_platelets_albumin', 
                     'haemoglobin_crp_platelets', 
                     'albumin_crp', 
                     'haemoglobin_crp', 
                     'platelets', 
                     'crp', 
                     'albumin', 
                     'haemoglobin']

test_group_label_abbrev = ['Hb+CRP+Plt+Alb', 'Hb+CRP+Plt', 'Alb+CRP', 'Hb+CRP', 'Plt', 'CRP', 'Alb', 'Hb']*3

In [163]:
age_test_ppv = age_test_ppv.reset_index().iloc[:, 1:]

In [ ]:
age_test_ppv

In [165]:
age_test_ppv_small = (
    age_test_ppv
    .query('test_group in @path_groups_small').reset_index().iloc[:, 1:].reset_index()
    .assign(study='Victoria')
    .assign(y=lambda df_: 24-df_.index.values)
    .assign(test_group_label_abbrev=test_group_label_abbrev)
)

In [166]:
# these are publicly available number's from Brian Nicholson's study
ppv_ci_l_bn = [1.79, 1.88, 2.13, 1.79, 2.13, 1.71, 1.28, 0.94, 
               12.37, 11.60, 10.49, 8.19, 6.91, 6.4, 8.96, 5.12, 
               7.51, 11.17, 4.95, 5.80, 4.95, 5.54, 4.35, 4.18]

ppv_ci_r_bn = [25.84, 23.03, 14.16, 9.21, 6.40, 4.86, 5.29, 2.99, 
               34.80, 31.81, 18.76, 13.73, 10.92, 9.55, 12.71, 6.91, 
               33.60, 30.96, 11.68, 11.26, 9.72, 9.04, 6.99, 5.54]

ppv_bn = [9.55, 8.61, 6.23, 4.43, 3.75, 2.90, 2.81, 1.79, 
          21.92, 20.21, 14.33, 10.58, 8.61, 7.85, 10.49, 5.71, 
          17.83, 16.89, 7.59, 8.10, 6.91, 6.99, 5.54, 4.95]

In [167]:
age_test_ppv_bn = (
    age_test_ppv_small
    .loc[:, ['index', 'test_group', 'age_group2_at_index_date']]
    .assign(ppv_ci_l=ppv_ci_l_bn)
    .assign(ppv_ci_r=ppv_ci_r_bn)
    .assign(ppv=ppv_bn)
    .assign(y=lambda df_: 24-df_.index.values+0.25)
    .assign(study='UK')
)

In [168]:
age_test_ppv_all = pd.concat([age_test_ppv_small, age_test_ppv_bn])

In [ ]:
age_test_ppv_all.query('study == "Victoria"')

In [ ]:
sns.set_style('white')

# change the labels of the groups
#ticklabels = age_test_ppv_small['age_group2_at_index_date'].str.replace('+', '').str.split('-').apply(lambda l: l[0])
ticklabels = age_test_ppv_small['age_group2_at_index_date'].astype(str).values + ' years, ' + age_test_ppv_small['test_group_label_abbrev'].astype(str).values

# plot these out
plt.figure(figsize=(10, 13))

# AL study
plt.hlines(y=age_test_ppv_all.query('study=="Victoria"').y, 
           xmin=age_test_ppv_all.query('study=="Victoria"').ppv_ci_l, 
           xmax=age_test_ppv_all.query('study=="Victoria"').ppv_ci_r, 
           color='r',
           alpha=0.5)

#sns.scatterplot(data=age_test_ppv_all.query('study=="AL"'), x='ppv_ci_l', y='y', color='k')
#sns.scatterplot(data=age_test_ppv_all.query('study=="AL"'), x='ppv_ci_r', y='y', color='k')
sns.scatterplot(data=age_test_ppv_all.query('study=="Victoria"'), x='ppv', y='y', color='r', label='AU')

# BN study
plt.hlines(y=age_test_ppv_all.query('study=="UK"').y, 
           xmin=age_test_ppv_all.query('study=="UK"').ppv_ci_l, 
           xmax=age_test_ppv_all.query('study=="UK"').ppv_ci_r, 
           color='b',
           alpha=0.5)

#sns.scatterplot(data=age_test_ppv_all.query('study=="BN"'), x='ppv_ci_l', y='y', color='k')
#sns.scatterplot(data=age_test_ppv_all.query('study=="BN"'), x='ppv_ci_r', y='y', color='k')
sns.scatterplot(data=age_test_ppv_all.query('study=="UK"'), x='ppv', y='y', color='b', label='UK')

plt.yticks(ticks=age_test_ppv_all.query('study=="Victoria"').y.values, labels=ticklabels)
plt.xlabel('Positive Predictive Values')
plt.ylabel('Age Group')
plt.xlim(-5, 60)
plt.legend()
plt.axvline(x=3, color='gray', linestyle='--', alpha=0.5, label='3% threshold')
plt.title('Positive predictive values of UWL patients by age group and pathology group')
plt.show()

### 1.4. Calculate by Age group, Gender, pathology test group

In [171]:
#cancer_counts_by_age_gender = patient_uwl_temp.sort_values(by=
#                                                           ['gender_code', 'age_group2']).groupby(['gender_code', 'age_group2'])['cancer'].sum().reset_index().rename(columns={'cancer': 'num_cancer'})
#patient_counts_by_age_gender = patient_uwl_temp.sort_values(by=
#                                                            ['gender_code', 'age_group2']).groupby(['gender_code', 'age_group2'])['cancer'].size().reset_index().rename(columns={'cancer': 'num_patients'})

In [41]:
# filter to a given path group
cancer_counts_by_age_gender

In [42]:
# filter to a given path group
patient_counts_by_age_gender

## Repeating these calculations for the 'Unintended' only encounter reasons

In [ ]:
temp_cohort = cohort[cohort['usi'].isin(usi_temp_cohort1)]
patient_uwl_temp = patient_uwl[patient_uwl['usi'].isin(usi_temp_cohort1)]

usi_cancer = temp_cohort[temp_cohort['cancer_within_6months_index_date'] == True]['usi'].unique()
usi_non_cancer = temp_cohort[temp_cohort['cancer_within_6months_index_date'] == False]['usi'].unique()

patient_uwl_temp['cancer'] = False
patient_uwl_temp.loc[patient_uwl_temp['usi'].isin(usi_cancer), 'cancer'] = True

cancer_counts_by_age_gender = patient_uwl_temp.sort_values(by=['gender_code', 'age_group']).groupby(['gender_code', 'age_group'])['cancer'].sum().reset_index().rename(columns={'cancer': 'num_cancer'})
patient_counts_by_age_gender = patient_uwl_temp.sort_values(by=['gender_code', 'age_group']).groupby(['gender_code', 'age_group'])['cancer'].size().reset_index().rename(columns={'cancer': 'num_patients'})

In [ ]:
# ppv: given that a patient has UWL, what is the probability of developing cancer within 6 months
temp_cohort[['usi', 'cancer_within_6months_index_date']].drop_duplicates()['cancer_within_6months_index_date'].value_counts()

In [ ]:
genders = ['Male', 'Female']

cancer_counts_by_age_gender = cancer_counts_by_age_gender.query('gender_code in @genders') 
patient_counts_by_age_gender =  patient_counts_by_age_gender.query('gender_code in @genders')

patient_counts_by_age_gender

In [ ]:
cohort_counts = cancer_counts_by_age_gender
cohort_counts['num_patients'] = patient_counts_by_age_gender['num_patients'].values

In [ ]:
# since interested in cumulative counts (number greater than a given age group) rather than less than
# we sort by reversed age order
cohort_counts = cohort_counts.sort_values(by=['gender_code', 'age_group'], 
                                          ascending=[True, False])

cohort_counts['num_no_cancer'] = cohort_counts['num_patients'] - cohort_counts['num_cancer']

# cumulative counts of cancer and non cancer patients
cohort_counts['num_cancer_cumulative'] = cohort_counts.sort_values(by=['gender_code', 'age_group'], 
                                                                   ascending=[True, False]).groupby('gender_code')['num_cancer'].cumsum()

cohort_counts['num_no_cancer_cumulative'] = cohort_counts.sort_values(by=['gender_code', 'age_group'], 
                                                                      ascending=[True, False]).groupby('gender_code')['num_no_cancer'].cumsum()

In [ ]:
# calculate the PPVs
cohort_ppv = pd.DataFrame(cohort_counts.apply(
    lambda r: calculate_ppv_with_ci(r['num_cancer_cumulative'], 
                                    r['num_no_cancer_cumulative']), axis=1).to_list(), 
             columns=['ppv', 'ppv_ci_l', 'ppv_ci_r'])*100

cohort_counts_final = pd.concat([cohort_counts.reset_index().iloc[:, 1:], cohort_ppv.reset_index().iloc[:, 1:]], axis=1)

cohort_counts_final = cohort_counts_final.sort_values(
    by=['gender_code', 'age_group'], ascending=[True, True]).reset_index().iloc[:, 1:]

cohort_counts_final['index'] = np.arange(cohort_counts_final.shape[0], 0, -1)

In [ ]:
cohort_counts_final